<a href="https://colab.research.google.com/github/harshitadd/AI4BharatTranslation/blob/main/LaBSEAligner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install scipy

In [ ]:
!pip install indic-nlp-library

In [14]:
import sys
from indicnlp.tokenize import indic_tokenize
from sentence_transformers import SentenceTransformer
import numpy as np
from scipy.spatial import distance
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
model = AutoModel.from_pretrained("sentence-transformers/LaBSE")

# labse_model, labse_layer = get_model(
#     model_url="https://tfhub.dev/google/LaBSE/1", max_seq_length=max_seq_length)

# LABSE_PATH = 'https://tfhub.dev/google/LaBSE/1'
# model = SentenceTransformer(LABSE_PATH,device='cpu')

# model = SentenceTransformer('sbert.net_models_LaBSE')


In [24]:
from sentence_transformers import SentenceTransformer
sentences = ["Hello World", "Hallo Welt"]

model = SentenceTransformer('LaBSE')


100%|██████████| 1.75G/1.75G [01:07<00:00, 25.9MB/s]


[[-0.07142267 -0.07716201 -0.03047759 ...  0.01356027 -0.04016103
  -0.02446148]
 [-0.06508803 -0.06923408 -0.03735013 ...  0.0101356  -0.04027328
  -0.02171571]]


In [39]:
embeddings = model.encode(sentences)
print(embeddings)

[[-0.07142267 -0.07716201 -0.03047759 ...  0.01356027 -0.04016103
  -0.02446148]
 [-0.06508803 -0.06923408 -0.03735013 ...  0.0101356  -0.04027328
  -0.02171571]]


In [ ]:
src = ["I am happy","I am sad", "I am excited", "I am folorn"]
tgtp = ["मैं खुश हूँ","मैं दुखी हूँ", "मैं उत्साहित हूं", "मैं मूर्ख हूं"]


In [33]:
def phrase_aligner(src, tgtp):
      '''
      This function is meant to align src phrases with best possible tgt phrase using LABSE model
      '''
      out = {}
      aligned_phrases = {}
      print("Performing phrase alignenment using LABSE")
      print(src,tgtp)
      src_phrases, tgt = src, tgtp
      
      for src_phrase in src_phrases:
          length_src_phrase = len(src_phrase.split())           
          tgt_token_list = split_tgt(length_src_phrase,tgt)
          embeddings_src_phrase, embeddings_tgt_tokens = generate_embeddings([src_phrase],tgt_token_list)
          alignments = get_target_sentence(embeddings_tgt_tokens, embeddings_src_phrase, length_src_phrase)
      
          if alignments is not None and alignments[2] is "MATCH":
              aligned_phrases[src_phrase] = tgt_token_list[alignments[0]]
          elif alignments is not None and alignments[2] is "NOMATCH": 
              print("No exact match found for:{} . Possible alignment {}".format(src_phrase,tgt_token_list[alignments[0]]))  
                      
      print("Aligned Phrases: {}".format(aligned_phrases))
      out = {"tgt":tgt,"src_phrases":src_phrases,"aligned_phrases":aligned_phrases}     

      # print("Error in LabseAlignerService:phrase_aligner: {} and {}".format(sys.exc_info()[0],e),MODULE_CONTEXT,e)
      # print("Error caught in LabseAlignerService:phrase_aligner for input: {}".format(inputs),MODULE_CONTEXT,e)
      
      return out

def split_tgt(length_src_phrase,tgt):
  tgt_token_list = list()
  tokenised_tgt = tgt.split()
  tgt_token_list = [tokenised_tgt[i:i+length_src_phrase] for i in range(len(tokenised_tgt)) if (i + length_src_phrase) <= len(tokenised_tgt)]
  tgt_token_list_plus = [tokenised_tgt[i:i+length_src_phrase+1] for i in range(len(tokenised_tgt)) if (i + length_src_phrase+1) <= len(tokenised_tgt)]
  tgt_token_list_minus = [tokenised_tgt[i:i+length_src_phrase-1] for i in range(len(tokenised_tgt)) if (i + length_src_phrase-1) <= len(tokenised_tgt) and 
                          length_src_phrase != 1]
  tgt_token_list = tgt_token_list + tgt_token_list_plus + tgt_token_list_minus
  tgt_token_list = [" ".join(j) for j in tgt_token_list]
  return tgt_token_list
      
def generate_embeddings(input_1, input_2):
  '''
  Generate LABSE embeddings
  Note: Inputs are array of strings
  '''           
  embeddings_input_1 = model.encode(input_1,show_progress_bar=True)
  embeddings_input_2 = model.encode(input_2,show_progress_bar=True)    
  print("LABSE embedding generation finished")
  return embeddings_input_1, embeddings_input_2
  
def get_target_sentence(target_embeddings, source_embedding, length_src_phrase):
  '''
  Calculate cosine similarity using scipy distance method
  '''
  distances = distance.cdist(source_embedding, target_embeddings, "cosine")[0]
  min_index = np.argmin(distances)
  min_distance = 1 - distances[min_index]
  print("Match score: {}".format(min_distance))
  if min_distance >= 0.5:
      return min_index, min_distance, "MATCH"
  else:
      return min_index, min_distance, "NOMATCH"     
      


In [53]:
src = "This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time. All amendments made by the Parliament up to and including the Constitution (One Hundred and Third Amendment) Act, 2019 are incorporated in this edition.The foot notes below the text indicate the Constitution Amendment Acts by which such amendments have been made."
src = src.split('.')
tgtp = "भारत का संविधान अप्रैल,  को यथाविद्यमान संसद्‌ द्वारा समयसमय पर यथा संशोधित भारत केसंविधान के पाठ को उद्धृत करता है । संसद्‌ द्वारा, संविधान (एक सौ तीनवां संशोधन) अधिनियम, तक और उसको सम्मिलित करते हुए, किए गए सभी संशोधन, इस संस्करण मैं सम्मिलित किए गएहै । पाठ के नीचे दिए गए पाद टिप्पण, संविधान संशोधन अधिनियमों, जिसके द्वारा ऐसे संशोधन किएगए है, को उपदर्शित करता है ।"

res = phrase_aligner(src, tgtp)

Performing phrase alignenment using LABSE
['This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time', ' All amendments made by the Parliament up to and including the Constitution (One Hundred and Third Amendment) Act, 2019 are incorporated in this edition', 'The foot notes below the text indicate the Constitution Amendment Acts by which such amendments have been made', ''] भारत का संविधान अप्रैल,  को यथाविद्यमान संसद्‌ द्वारा समयसमय पर यथा संशोधित भारत केसंविधान के पाठ को उद्धृत करता है । संसद्‌ द्वारा, संविधान (एक सौ तीनवां संशोधन) अधिनियम, तक और उसको सम्मिलित करते हुए, किए गए सभी संशोधन, इस संस्करण मैं सम्मिलित किए गएहै । पाठ के नीचे दिए गए पाद टिप्पण, संविधान संशोधन अधिनियमों, जिसके द्वारा ऐसे संशोधन किएगए है, को उपदर्शित करता है ।



LABSE embedding generation finished
Match score: 0.7694120117489679



LABSE embedding generation finished
Match score: 0.8732949153322594



LABSE embedding generation finished
Match score: 0.8812351099551589



LABSE embedding generation finished
Match score: 0.9999999999998788
Aligned Phrases: {'This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time': 'को यथाविद्यमान संसद्\u200c द्वारा समयसमय पर यथा संशोधित भारत केसंविधान के पाठ को उद्धृत करता है । संसद्\u200c द्वारा, संविधान (एक सौ', ' All amendments made by the Parliament up to and including the Constitution (One Hundred and Third Amendment) Act, 2019 are incorporated in this edition': 'है । संसद्\u200c द्वारा, संविधान (एक सौ तीनवां संशोधन) अधिनियम, तक और उसको सम्मिलित करते हुए, किए गए सभी संशोधन, इस संस्करण मैं सम्मिलित', 'The foot notes below the text indicate the Constitution Amendment Acts by which such amendments have been made': 'पाठ के नीचे दिए गए पाद टिप्पण, संविधान संशोधन अधिनियमों, जिसके द्वारा ऐसे संशोधन किएगए है, को उपदर्शित करता', '': ''}


In [57]:
print(res)

{'tgt': 'भारत का संविधान अप्रैल,  को यथाविद्यमान संसद्\u200c द्वारा समयसमय पर यथा संशोधित भारत केसंविधान के पाठ को उद्धृत करता है । संसद्\u200c द्वारा, संविधान (एक सौ तीनवां संशोधन) अधिनियम, तक और उसको सम्मिलित करते हुए, किए गए सभी संशोधन, इस संस्करण मैं सम्मिलित किए गएहै । पाठ के नीचे दिए गए पाद टिप्पण, संविधान संशोधन अधिनियमों, जिसके द्वारा ऐसे संशोधन किएगए है, को उपदर्शित करता है ।', 'src_phrases': ['This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time', ' All amendments made by the Parliament up to and including the Constitution (One Hundred and Third Amendment) Act, 2019 are incorporated in this edition', 'The foot notes below the text indicate the Constitution Amendment Acts by which such amendments have been made', ''], 'aligned_phrases': {'This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time': 'को यथाविद्यमान संसद्\u20

In [55]:
print(res['aligned_phrases'])

{'This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time': 'को यथाविद्यमान संसद्\u200c द्वारा समयसमय पर यथा संशोधित भारत केसंविधान के पाठ को उद्धृत करता है । संसद्\u200c द्वारा, संविधान (एक सौ', ' All amendments made by the Parliament up to and including the Constitution (One Hundred and Third Amendment) Act, 2019 are incorporated in this edition': 'है । संसद्\u200c द्वारा, संविधान (एक सौ तीनवां संशोधन) अधिनियम, तक और उसको सम्मिलित करते हुए, किए गए सभी संशोधन, इस संस्करण मैं सम्मिलित', 'The foot notes below the text indicate the Constitution Amendment Acts by which such amendments have been made': 'पाठ के नीचे दिए गए पाद टिप्पण, संविधान संशोधन अधिनियमों, जिसके द्वारा ऐसे संशोधन किएगए है, को उपदर्शित करता', '': ''}


In [60]:
for key in res['aligned_phrases']:
  print(key)
  print(res['aligned_phrases'][key])
  

This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time
को यथाविद्यमान संसद्‌ द्वारा समयसमय पर यथा संशोधित भारत केसंविधान के पाठ को उद्धृत करता है । संसद्‌ द्वारा, संविधान (एक सौ
 All amendments made by the Parliament up to and including the Constitution (One Hundred and Third Amendment) Act, 2019 are incorporated in this edition
है । संसद्‌ द्वारा, संविधान (एक सौ तीनवां संशोधन) अधिनियम, तक और उसको सम्मिलित करते हुए, किए गए सभी संशोधन, इस संस्करण मैं सम्मिलित
The foot notes below the text indicate the Constitution Amendment Acts by which such amendments have been made
पाठ के नीचे दिए गए पाद टिप्पण, संविधान संशोधन अधिनियमों, जिसके द्वारा ऐसे संशोधन किएगए है, को उपदर्शित करता


